# 🏆 Week 8: Re-ranking

**Learning Objectives:**
1. Understand why re-ranking improves retrieval
2. Implement cross-encoder re-ranker
3. Evaluate before/after re-ranking
4. Optimize latency vs quality trade-offs

---

In [ ]:
import numpy as np
from typing import List, Tuple

---
# Section 1: Theory
---

## Why Re-ranking?

| Stage | Model | Speed | Quality |
|-------|-------|-------|--------|
| Retrieval | Bi-encoder | Fast | Good |
| Re-ranking | Cross-encoder | Slow | Excellent |

**Two-stage approach:**
1. Fast retrieval: Get top-100 candidates
2. Slow re-ranking: Score top-100 → return top-10

---
# Section 2: Hands-On Implementation
---

In [ ]:
class SimpleCrossEncoder:
    """Simulated cross-encoder for demo."""
    
    def __init__(self):
        # Simulated model weights
        self.keyword_boost = 0.3
    
    def score(self, query: str, document: str) -> float:
        """Score query-document pair."""
        # Simulate cross-encoder scoring
        query_words = set(query.lower().split())
        doc_words = set(document.lower().split())
        
        # Overlap score
        overlap = len(query_words & doc_words)
        
        # Length normalization
        norm = len(query_words) + len(doc_words)
        
        # Add some randomness for realism
        np.random.seed(hash(query + document) % 2**32)
        noise = np.random.uniform(-0.1, 0.1)
        
        return (overlap / norm * 2) + self.keyword_boost + noise
    
    def rerank(self, query: str, documents: List[str], top_k: int = 5) -> List[Tuple[int, float, str]]:
        """Re-rank documents by relevance to query."""
        scores = []
        for i, doc in enumerate(documents):
            score = self.score(query, doc)
            scores.append((i, score, doc))
        
        scores.sort(key=lambda x: x[1], reverse=True)
        return scores[:top_k]

In [ ]:
# Test re-ranking
documents = [
    "Python programming for beginners",
    "Machine learning with Python tutorial",
    "Deep learning neural networks",
    "Python data science handbook",
    "Natural language processing basics"
]

# Initial retrieval order (simulated)
initial_order = [2, 4, 0, 1, 3]  # Suboptimal order

reranker = SimpleCrossEncoder()
query = "Python machine learning"

print(f"Query: '{query}'\n")
print("Initial Order:")
for i, idx in enumerate(initial_order):
    print(f"  {i+1}. {documents[idx]}")

# Re-rank
retrieved_docs = [documents[i] for i in initial_order]
reranked = reranker.rerank(query, retrieved_docs, top_k=5)

print("\nAfter Re-ranking:")
for i, (orig_idx, score, doc) in enumerate(reranked):
    print(f"  {i+1}. [{score:.3f}] {doc}")

## 2.2 NDCG Evaluation

In [ ]:
def dcg(relevances, k):
    """Discounted Cumulative Gain."""
    return sum(rel / np.log2(i + 2) for i, rel in enumerate(relevances[:k]))

def ndcg(retrieved_relevances, ideal_relevances, k):
    """Normalized DCG."""
    ideal_sorted = sorted(ideal_relevances, reverse=True)
    idcg = dcg(ideal_sorted, k)
    if idcg == 0:
        return 0
    return dcg(retrieved_relevances, k) / idcg

# Example: relevance scores (3=highly relevant, 0=not relevant)
before_rerank = [0, 1, 3, 2, 1]  # Bad order
after_rerank = [3, 2, 1, 1, 0]   # Good order
ideal = [3, 2, 1, 1, 0]

print(f"NDCG@5 before re-ranking: {ndcg(before_rerank, ideal, 5):.4f}")
print(f"NDCG@5 after re-ranking: {ndcg(after_rerank, ideal, 5):.4f}")

---
# Section 3: Unit Tests
---

In [ ]:
def run_tests():
    print("Running Unit Tests...\n")
    
    # Test cross-encoder
    ce = SimpleCrossEncoder()
    score = ce.score("python", "python programming")
    assert score > 0
    print("✓ Cross-encoder scoring test passed")
    
    # Test NDCG perfect score
    assert abs(ndcg([3, 2, 1], [3, 2, 1], 3) - 1.0) < 0.01
    print("✓ NDCG perfect score test passed")
    
    print("\n🎉 All tests passed!")

run_tests()

---
# Section 4: Interview Prep
---

### Q1: Bi-encoder vs Cross-encoder?
**Answer:** Bi-encoder: encodes query/doc separately, fast. Cross-encoder: encodes together, slow but more accurate.

### Q2: How to optimize re-ranking latency?
**Answer:** Limit candidates (top-50), use distillation, batch processing, caching.

---
# Section 5: Deliverable
---

**Created:** `reranker_demo.py`

**Next Week:** LLM Orchestration & Streaming